In [515]:
"""
References:
https://arxiv.org/pdf/1512.03385.pdf
https://arxiv.org/pdf/1604.04112v4.pdf
https://github.com/yu4u/cutout-random-erasing
"""
import keras
import numpy as np
from keras.layers import Add, Dense, Conv2D, BatchNormalization
from keras.layers import Activation, AveragePooling2D, Input, Flatten
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.models import Model

In [516]:
batch_size = 128
epochs = 180
n_classes = 10
learning_rate = 0.1

In [517]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [518]:
class Resnet:
    def __init__(self, size=44, stacks=3, starting_filter=16):
        self.size = size
        self.stacks = stacks
        self.starting_filter = starting_filter
        self.residual_blocks = (size - 2) // 6
        
    def get_model(self, input_shape=(32, 32, 3), n_classes=10):
        n_filters = self.starting_filter

        inputs = Input(shape=input_shape)
        network = self.layer(inputs, n_filters)
        network = self.stack(network, n_filters, True)

        for _ in range(self.stacks - 1):
            n_filters *= 2
            network = self.stack(network, n_filters)

        network = Activation('elu')(network)
        network = AveragePooling2D(pool_size=network.shape[1])(network)
        network = Flatten()(network)
        outputs = Dense(n_classes, activation='softmax', 
                        kernel_initializer='he_normal')(network)

        model = Model(inputs=inputs, outputs=outputs)

        return model
    
    def stack(self, inputs, n_filters, first_stack=False):
        stack = inputs

        if first_stack:
            stack = self.identity_block(stack, n_filters)
        else:
            stack = self.convolution_block(stack, n_filters)

        for _ in range(self.residual_blocks - 1):
            stack = self.identity_block(stack, n_filters)

        return stack
    
    def identity_block(self, inputs, n_filters):
        shortcut = inputs

        block = self.layer(inputs, n_filters, normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        block = Add()([shortcut, block])

        return block

    def convolution_block(self, inputs, n_filters, strides=2):
        shortcut = inputs

        block = self.layer(inputs, n_filters, strides=strides,
                           normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        shortcut = self.layer(shortcut, n_filters,
                              kernel_size=1, strides=strides,
                              activation=None)

        block = Add()([shortcut, block])

        return block
    
    def layer(self, inputs, n_filters, kernel_size=3,
              strides=1, activation='elu', normalize_batch=True):
    
        convolution = Conv2D(n_filters, kernel_size=kernel_size,
                             strides=strides, padding='same',
                             kernel_initializer="he_normal",
                             kernel_regularizer=l2(1e-4))

        x = convolution(inputs)

        if normalize_batch:
            x = BatchNormalization()(x)

        if activation is not None:
            x = Activation(activation)(x)

        return x
    
def learning_rate_schedule(epoch):
    new_learning_rate = learning_rate

    if epoch <= 90:
        pass
    elif epoch > 90 and epoch <= 140:
        new_learning_rate = learning_rate * 0.1
    else:
        new_learning_rate = learning_rate * 0.01
        
    print('Learning rate:', new_learning_rate)
    
    return new_learning_rate

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        if input_img.ndim == 3:
            img_h, img_w, img_c = input_img.shape
        elif input_img.ndim == 2:
            img_h, img_w = input_img.shape

        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            if input_img.ndim == 3:
                c = np.random.uniform(v_l, v_h, (h, w, img_c))
            if input_img.ndim == 2:
                c = np.random.uniform(v_l, v_h, (h, w))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w] = c

        return input_img

    return eraser

In [519]:
resnet = Resnet()

model = resnet.get_model()

optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "model_45"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_46 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_2013 (Conv2D)            (None, 32, 32, 16)   448         input_46[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1462 (Batch (None, 32, 32, 16)   64          conv2d_2013[0][0]                
__________________________________________________________________________________________________
activation_1429 (Activation)    (None, 32, 32, 16)   0           batch_normalization_1462[0][0]   
___________________________________________________________________________________________

In [520]:
lr_scheduler = LearningRateScheduler(learning_rate_schedule)
callbacks = [lr_scheduler]

datagen = ImageDataGenerator(width_shift_range=4,
                             height_shift_range=4,
                             horizontal_flip=True,
                             preprocessing_function=get_random_eraser(pixel_level=True))
datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          validation_data=(x_test, y_test),
          epochs=epochs, workers=4,
          callbacks=callbacks)

model.save('model.h5')

Epoch 1/180
Learning rate: 0.1
391/391 [==============================] - 24s 54ms/step - loss: 1.9110 - accuracy: 0.4273 - val_loss: 1.5964 - val_accuracy: 0.5440
Epoch 2/180
Learning rate: 0.1
391/391 [==============================] - 23s 58ms/step - loss: 1.4496 - accuracy: 0.5967 - val_loss: 1.4109 - val_accuracy: 0.6202
Epoch 3/180
Learning rate: 0.1
391/391 [==============================] - 23s 58ms/step - loss: 1.2316 - accuracy: 0.6638 - val_loss: 1.1242 - val_accuracy: 0.7089
Epoch 4/180
Learning rate: 0.1
391/391 [==============================] - 24s 61ms/step - loss: 1.1036 - accuracy: 0.7088 - val_loss: 1.0586 - val_accuracy: 0.7300
Epoch 5/180
Learning rate: 0.1
391/391 [==============================] - 23s 60ms/step - loss: 1.0161 - accuracy: 0.7370 - val_loss: 0.9467 - val_accuracy: 0.7602
Epoch 6/180
Learning rate: 0.1
391/391 [==============================] - 23s 58ms/step - loss: 0.9413 - accuracy: 0.7604 - val_loss: 0.8926 - val_accuracy: 0.7728
Epoch 7/180
Lear

391/391 [==============================] - 23s 59ms/step - loss: 0.5682 - accuracy: 0.8911 - val_loss: 0.6654 - val_accuracy: 0.8648
Epoch 51/180
Learning rate: 0.1
391/391 [==============================] - 23s 59ms/step - loss: 0.5696 - accuracy: 0.8915 - val_loss: 0.6823 - val_accuracy: 0.8644
Epoch 52/180
Learning rate: 0.1
391/391 [==============================] - 23s 58ms/step - loss: 0.5705 - accuracy: 0.8923 - val_loss: 0.6453 - val_accuracy: 0.8775
Epoch 53/180
Learning rate: 0.1
391/391 [==============================] - 23s 58ms/step - loss: 0.5681 - accuracy: 0.8947 - val_loss: 0.6352 - val_accuracy: 0.8745
Epoch 54/180
Learning rate: 0.1
391/391 [==============================] - 23s 58ms/step - loss: 0.5691 - accuracy: 0.8936 - val_loss: 0.6185 - val_accuracy: 0.8846
Epoch 55/180
Learning rate: 0.1
391/391 [==============================] - 23s 59ms/step - loss: 0.5685 - accuracy: 0.8954 - val_loss: 0.5977 - val_accuracy: 0.8963
Epoch 56/180
Learning rate: 0.1
391/391 [=

391/391 [==============================] - 24s 61ms/step - loss: 0.3500 - accuracy: 0.9647 - val_loss: 0.4476 - val_accuracy: 0.9369
Epoch 100/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 24s 60ms/step - loss: 0.3479 - accuracy: 0.9637 - val_loss: 0.4457 - val_accuracy: 0.9375
Epoch 101/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 24s 60ms/step - loss: 0.3406 - accuracy: 0.9658 - val_loss: 0.4385 - val_accuracy: 0.9365
Epoch 102/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 24s 60ms/step - loss: 0.3330 - accuracy: 0.9682 - val_loss: 0.4384 - val_accuracy: 0.9370
Epoch 103/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 23s 59ms/step - loss: 0.3277 - accuracy: 0.9679 - val_loss: 0.4331 - val_accuracy: 0.9371
Epoch 104/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 24s 62ms/step - loss: 0.3207 - accuracy: 0

391/391 [==============================] - 23s 58ms/step - loss: 0.1923 - accuracy: 0.9825 - val_loss: 0.3526 - val_accuracy: 0.9447
Epoch 145/180
Learning rate: 0.001
391/391 [==============================] - 23s 58ms/step - loss: 0.1933 - accuracy: 0.9830 - val_loss: 0.3515 - val_accuracy: 0.9446
Epoch 146/180
Learning rate: 0.001
391/391 [==============================] - 23s 58ms/step - loss: 0.1885 - accuracy: 0.9836 - val_loss: 0.3488 - val_accuracy: 0.9448
Epoch 147/180
Learning rate: 0.001
391/391 [==============================] - 23s 58ms/step - loss: 0.1884 - accuracy: 0.9842 - val_loss: 0.3498 - val_accuracy: 0.9451
Epoch 148/180
Learning rate: 0.001
391/391 [==============================] - 23s 58ms/step - loss: 0.1902 - accuracy: 0.9831 - val_loss: 0.3477 - val_accuracy: 0.9458
Epoch 149/180
Learning rate: 0.001
391/391 [==============================] - 23s 58ms/step - loss: 0.1887 - accuracy: 0.9834 - val_loss: 0.3472 - val_accuracy: 0.9466
Epoch 150/180
Learning rate